In [ ]:
import pandas as pd
import uproot
import numpy as np

# reading root file
root_file = uproot.open('../build/output0.root')
photons = root_file["Photons"]

In [ ]:
# Considering only optical photons
photonType = [t for t in photons["fType"].array()]
processType = [t for t in photons["fProcess"].array()]
# status = [t for t in photons["fStepStatusNumber"].array()]
# processType2 = [t for t in photons["fProcessType"].array()]
# subSrocessType = [t for t in photons["fSubProcessType"].array()]
creatorProcess = [t for t in photons["fCreatorProcess"].array()]
trackID = [t for t in photons["ftrackID"].array()]
parentTrackID = [t for t in photons["fParentTrackID"].array()]
material = [t for t in photons["fMaterial"].array()]
relevant_ind = [t == 'Scintillation' for t in creatorProcess]
photonsX = np.array(photons["fX"].array())[relevant_ind]
photonsY = np.array(photons["fY"].array())[relevant_ind]
photonsZ = np.array(photons["fZ"].array())[relevant_ind]

trackToParticle_dict = {}
for particle, t_id in zip(photonType, trackID):
    trackToParticle_dict[t_id] = particle
parentParticle = [trackToParticle_dict[t_id] if t_id in trackID else "none" for t_id in parentTrackID]

def compute_hist(l):
    d = {}
    for t in l:
        if t not in d:
            d[t] = 1
        else:
            d[t] += 1
    return d

jointDict = compute_hist([p1 + ', ' + p2 + ', ' + str(p3) + ', ' + str(p4) + ', ' + str(p5)
                          for (((p1,p2), p3), p4), p5 in zip(zip(zip(zip(photonType, processType), material), parentParticle), creatorProcess)])
print("particle\tproces\t\tmaterial\tparentPar\tcreatorProc\tnum events\n")
for key, item in jointDict.items():
    line = '\t\t'.join([k[:6] for k in key.split(', ')]) + '\t\t' + str(item)
    print(line)

In [ ]:
# Plot absorption as a function of the depth
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
fig = plt.figure()
ax = fig.add_subplot(111)

ax.hist(photonsZ*1e6, bins=int(np.sqrt(len(photonsZ))))

ax.set_xlabel('Z [nm]')
ax.set_ylabel('# of photons created')
ax.set_title('Histogram of photons depth creation')
plt.show()

In [ ]:
from scipy.stats import gaussian_kde
import multiprocessing
import os
os.environ['DISPLAY'] = 'localhost:10.0'

def calc_kde(data):
    return kde(data.T)

# Calculate kernel density estimation
photonsX_kde, photonsY_kde, photonsZ_kde = photonsX*1e6, photonsY*1e6, photonsZ*1e6
kde = gaussian_kde(np.row_stack((photonsX_kde, photonsY_kde, photonsZ_kde)))

# Evaluate kde on a grid
grid_size = 30j
factor = 30
xmin, ymin, zmin = photonsX_kde.min()/factor, photonsY_kde.min()/factor, photonsZ_kde.min()
xmax, ymax, zmax = photonsX_kde.max()/factor, photonsY_kde.max()/factor, photonsZ_kde.max()
xi, yi, zi = np.mgrid[xmin:xmax:grid_size, ymin:ymax:grid_size, zmin:zmax:grid_size]
coords = np.vstack([item.ravel() for item in [xi, yi, zi]]) 

# Multiprocessing
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
results = pool.map(calc_kde, np.array_split(coords.T, 2))
density = np.concatenate(results).reshape(xi.shape)

In [ ]:
# Plot scatter with mayavi
from mayavi import mlab
figure = mlab.figure('DensityPlot', bgcolor=(1,1,1), fgcolor=(0,0,0))

grid = mlab.pipeline.scalar_field(xi, yi, zi, density)
mlab.pipeline.volume(grid, vmin=density.min() + .5*(density.max()-density.min()), vmax=density.min() + .9*(density.max()-density.min()))

mlab.axes()
mlab.show()

In [ ]:
import os
os.environ['DISPLAY'] = 'localhost:10.0'
# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Set the transparency as a function of density
alpha = ((density - density.min()) / (density.max() - density.min()))**100

# Scatter plot the points with color based on density
ax.scatter(xi, yi, zi, c=density.flatten(), alpha=alpha.flatten(), cmap=cm.coolwarm)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Density of Photon Creation')
plt.show()